# Constucturing The Model

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.layers import Dense, Flatten
import pandas as pd


In [9]:
height,width=180,180



## Recreate the model and load the weights

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.legacy import Adam

# Load the EfficientNetV2S model without the top layer
base_model = tf.keras.applications.EfficientNetV2S(
    include_top=False,  # Do not include the fully connected layer at the top
    weights="imagenet",  # Use weights pre-trained on ImageNet
    input_tensor=None,  # Optional custom input tensor
    input_shape=None,  # Specify your custom input shape if necessary
    pooling=None,  # Specify this if you need a specific type of pooling
    include_preprocessing=True  # Include preprocessing according to the model's requirements
)

# Make sure the base model is not trainable
base_model.trainable = False

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer for 10 classes
predictions = Dense(10, activation='softmax')(x)

# This is the model we will train
efficient_net_new = Model(inputs=base_model.input, outputs=predictions)

efficient_net_new.compile(optimizer=Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

efficient_net_new.load_weights('efficientnet_augmentation_full_dataset_epochs30.h5')

/Users/EdenChung/anaconda3/envs/ml/lib/python3.9/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## Create the submission

In [6]:
##Creating the submission

height = 384
width = 384

def preprocess_image(image_path, height, width):
    img = image.load_img(image_path, target_size=(height, width))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch dimension
    resized_image = tf.image.resize(img_array, [384, 384])  # Resize to the expected input size
    return resized_image

#test_df = pd.read_csv('test.csv')
test_df = pd.read_csv('../../BTTAIxNYBG-test.csv')

# Initialize a list to store predictions
predictions_list = []

for _, row in test_df.iterrows():
    unique_id = row['uniqueID']
    file_name = row['imageFile']
    
    # Load and preprocess the image
    img_path = os.path.join('../../test', file_name)  # Update with your test images path
    img_array = preprocess_image(img_path, height, width)
    
    # Predict
    predictions = efficient_net_new.predict(img_array)
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]  # Take the argmax to get the predicted class ID
    
    # Store the uniqueID and predicted classID
    predictions_list.append({'uniqueID': unique_id, 'classID': predicted_class})

# Convert the predictions list to a DataFrame
predictions_df = pd.DataFrame(predictions_list)

# Save the predictions to a new CSV file
predictions_df.to_csv('predictions.csv', index=False)


1/1 [==============================] - 0s 144ms/step
